# Import All Modules

In [34]:
# Enable Intellisense for code assisatance and autocomplete
%config IPCompleter.greedy=True

#Install standard modules
import pandas as pd
import numpy as np

# Time modules
import datetime as dt
import datetime
from datetime import timedelta, date, datetime
from dateutil.relativedelta import relativedelta
import time

# Progress bar modules
from tqdm.notebook import tqdm

# API modules
import requests


In [35]:
# Input AMeritrade API key
api_key = open("Ameritrade_API_key.txt", "r").read()

# Function to check weekly options

In [36]:
# Define empty dictionary
interesting_trades  = {}
tickers_without_weekly = []
tickers_with_weekly = []

# Define function to check for weekly options
def weekly_option_chain_checker(ticker):
    
    # get today's date and date in 2 weeks
    todays_date = date.today()
    end_date = todays_date + timedelta(days=14)

     # define our endpoint
    endpoint = r"https://api.tdameritrade.com/v1/marketdata/chains"

     # define our payload
    payload = {'apikey' : api_key,
               'symbol' : ticker,
               'contractType':'ALL',
               'includeQuotes':'FALSE',
               'strategy':'SINGLE',
               'strikeCount' : '1',
               'fromDate':todays_date,
               'toDate':end_date,
               'optionType':'S'
               }

     # make a request
    content = requests.get(url = endpoint, params = payload)

    # convert it to dictionary
    data = content.json()
    
    if data['status'] == 'FAILED':
        pass
    else:        
        # Check if there are less then 4 put and call options in the next two weeks 
        # to check if the options are weekly
        if data["numberOfContracts"] < 4:
            tickers_without_weekly.append(ticker)
        else:
            # Append ticker to list for next function
            tickers_with_weekly.append(ticker)      
        
    # wait 0.5 second   
    time.sleep(0.5)
    return

# Check for which stocks have weekly Options

In [37]:
# List of all stocks
stocks_df = pd.read_csv("list_of_all_stocks.csv",index_col=0)
list_all_tickers = stocks_df['ticker']
list_all_tickers = np.unique(list_all_tickers)

In [ ]:
# run the function to check tickers for weekly options and append into a new list
for i in tqdm(list(list_all_tickers)):
    try:
        weekly_option_chain_checker(i)
    except:
        continue
    

### Save data from weekly check to CSV

In [ ]:
weekly_stocks_to_check = pd.DataFrame(tickers_with_weekly)
weekly_stocks_to_check.to_csv('weekly_stocks_to_check.csv')